# Phase 3: Predictive Modeling

This notebook builds machine learning models to predict:
1. **Next Station Arrival Time**: When will a group reach the next station?
2. **Total Completion Time**: How long until a group finishes all stations?

## Prerequisites:
- Complete Phase 1 (Station Detection)
- Complete Phase 2 (Temporal Analysis)
- Phase 1 & 2 outputs must exist in respective directories

## Objectives:
- Engineer features from temporal data
- Train Random Forest model for next station prediction
- Train Gradient Boosting model for completion time prediction
- Evaluate model performance

## Output:
- Trained models
- Prediction results
- Feature importance analysis

## Workshop Selection

In [ ]:
# ============================================
# WORKSHOP SELECTION
# ============================================
# Must match the workshop used in Phase 1 & 2
# Valid options: "Workshop1", "Workshop2", "Workshop3"

WORKSHOP = "Workshop2"  # 👈 CHANGE THIS VALUE

# ============================================

print(f"🎯 Selected Workshop: {WORKSHOP}")
print(f"{'='*50}")
print(f"Loading Phase 1 & 2 results for {WORKSHOP}...")
print(f"{'='*50}\n")

## Setup and Load Previous Phase Results

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from pathlib import Path
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Libraries imported successfully!")

In [ ]:
# Load Phase 1 results
phase1_dir = Path(f'../data/phase1_results/{WORKSHOP}')
phase2_dir = Path(f'../data/phase2_results/{WORKSHOP}')

# Check if previous phases exist
if not phase1_dir.exists():
    raise FileNotFoundError(
        f"Phase 1 results not found for {WORKSHOP}.\n"
        f"Please run phase1_station_detection.ipynb first!"
    )

if not phase2_dir.exists():
    raise FileNotFoundError(
        f"Phase 2 results not found for {WORKSHOP}.\n"
        f"Please run phase2_temporal_analysis.ipynb first!"
    )

# Load station centroids from Phase 1
station_info = pd.read_csv(phase1_dir / 'station_centroids.csv')

# Load visits data from Phase 2
visits_df = pd.read_csv(phase2_dir / 'station_visits.csv')
visits_df['start_time'] = pd.to_datetime(visits_df['start_time'])
visits_df['end_time'] = pd.to_datetime(visits_df['end_time'])

# Load travel times from Phase 2
travel_df = pd.read_csv(phase2_dir / 'travel_times.csv')

print(f"\n{'='*60}")
print(f"📊 Loaded Previous Phase Results for {WORKSHOP}")
print(f"{'='*60}")
print(f"Stations: {len(station_info)}")
print(f"Station visits: {len(visits_df)}")
print(f"Transitions: {len(travel_df)}")
print(f"Groups: {sorted(visits_df['group'].unique())}")
print(f"{'='*60}\n")

## 3.1 Feature Engineering for Prediction

In [ ]:
# Create features for predicting next station arrival
prediction_data = []

for group, group_visits in visits_df.groupby('group'):
    group_visits = group_visits.sort_values('start_time').reset_index(drop=True)
    
    for i in range(len(group_visits) - 1):
        current = group_visits.iloc[i]
        next_visit = group_visits.iloc[i + 1]
        
        # Calculate time to next station
        time_to_next = (next_visit['start_time'] - current['end_time']).total_seconds() / 60
        
        # Get centroid coordinates
        current_centroid = station_info[station_info['station'] == current['station']].iloc[0]
        next_centroid = station_info[station_info['station'] == next_visit['station']].iloc[0]
        
        # Calculate Euclidean distance
        distance = np.sqrt(
            (next_centroid['centroid_x'] - current_centroid['centroid_x'])**2 + 
            (next_centroid['centroid_y'] - current_centroid['centroid_y'])**2
        )
        
        prediction_data.append({
            'group': group,
            'current_station': current['station'],
            'next_station': next_visit['station'],
            'visit_number': i,  # Which visit in sequence
            'current_dwell_time': current['duration_minutes'],
            'distance_to_next': distance,
            'time_of_day_hour': current['end_time'].hour + current['end_time'].minute / 60,
            'elapsed_time_minutes': (current['end_time'] - group_visits.iloc[0]['start_time']).total_seconds() / 60,
            'avg_prev_dwell': group_visits.iloc[:i+1]['duration_minutes'].mean() if i > 0 else current['duration_minutes'],
            'target_time_to_next': time_to_next  # Target variable
        })

pred_df = pd.DataFrame(prediction_data)

print(f"Prediction dataset size: {len(pred_df)}")
print(f"\nFeatures: {[col for col in pred_df.columns if col != 'target_time_to_next']}")
print(f"\nTarget variable: target_time_to_next (minutes to reach next station)")

pred_df.head(10)

In [ ]:
# Check correlations
numeric_cols = ['visit_number', 'current_dwell_time', 'distance_to_next', 
                'time_of_day_hour', 'elapsed_time_minutes', 'avg_prev_dwell', 'target_time_to_next']

plt.figure(figsize=(10, 8))
sns.heatmap(pred_df[numeric_cols].corr(), annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title(f'{WORKSHOP}: Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 3.2 Model 1: Predict Time to Next Station

In [ ]:
# Prepare training data
feature_cols = ['current_station', 'next_station', 'visit_number', 'current_dwell_time', 
                'distance_to_next', 'time_of_day_hour', 'elapsed_time_minutes', 'avg_prev_dwell']

X = pred_df[feature_cols]
y = pred_df['target_time_to_next']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

In [ ]:
# Train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Predictions
y_pred_train = rf_model.predict(X_train)
y_pred_test = rf_model.predict(X_test)

# Evaluate
print(f"{'='*60}")
print(f"Random Forest - Time to Next Station Prediction")
print(f"{'='*60}")
print(f"Training Set:")
print(f"  MAE: {mean_absolute_error(y_train, y_pred_train):.2f} minutes")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f} minutes")
print(f"  R²: {r2_score(y_train, y_pred_train):.3f}")

print(f"\nTest Set:")
print(f"  MAE: {mean_absolute_error(y_test, y_pred_test):.2f} minutes")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f} minutes")
print(f"  R²: {r2_score(y_test, y_pred_test):.3f}")
print(f"{'='*60}")

In [ ]:
# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'], color='teal')
plt.xlabel('Importance', fontsize=12)
plt.title(f'{WORKSHOP}: Feature Importance for Next Station Arrival', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\nTop Features:")
print(feature_importance)

In [ ]:
# Visualize predictions vs actual
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot
axes[0].scatter(y_test, y_pred_test, alpha=0.5, s=30)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Time to Next Station (min)', fontsize=11)
axes[0].set_ylabel('Predicted Time (min)', fontsize=11)
axes[0].set_title('Predictions vs Actual', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Residual plot
residuals = y_test - y_pred_test
axes[1].scatter(y_pred_test, residuals, alpha=0.5, s=30)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Time (min)', fontsize=11)
axes[1].set_ylabel('Residuals (min)', fontsize=11)
axes[1].set_title('Residual Plot', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3.3 Model 2: Predict Total Completion Time

In [ ]:
# Calculate total time for each group to complete all stations
completion_data = []

for group, group_visits in visits_df.groupby('group'):
    group_visits = group_visits.sort_values('start_time')
    
    if len(group_visits) < 2:
        continue
    
    start_time = group_visits.iloc[0]['start_time']
    end_time = group_visits.iloc[-1]['end_time']
    total_time = (end_time - start_time).total_seconds() / 60
    
    completion_data.append({
        'group': group,
        'total_stations_visited': len(group_visits),
        'unique_stations': group_visits['station'].nunique(),
        'avg_dwell_time': group_visits['duration_minutes'].mean(),
        'total_dwell_time': group_visits['duration_minutes'].sum(),
        'first_station': group_visits.iloc[0]['station'],
        'start_hour': start_time.hour + start_time.minute / 60,
        'total_completion_time': total_time  # Target
    })

completion_df = pd.DataFrame(completion_data)

print(f"Groups analyzed: {len(completion_df)}")
print(f"\nCompletion Time Statistics:")
print(completion_df['total_completion_time'].describe())

completion_df.head()

In [ ]:
# Prepare data for completion time prediction
completion_features = ['total_stations_visited', 'unique_stations', 'avg_dwell_time', 
                       'first_station', 'start_hour']

X_comp = completion_df[completion_features]
y_comp = completion_df['total_completion_time']

# Split
X_train_comp, X_test_comp, y_train_comp, y_test_comp = train_test_split(
    X_comp, y_comp, test_size=0.2, random_state=42
)

# Train Gradient Boosting model
gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
gb_model.fit(X_train_comp, y_train_comp)

# Predictions
y_pred_train_comp = gb_model.predict(X_train_comp)
y_pred_test_comp = gb_model.predict(X_test_comp)

# Evaluate
print(f"{'='*60}")
print(f"Gradient Boosting - Total Completion Time Prediction")
print(f"{'='*60}")
print(f"Training Set:")
print(f"  MAE: {mean_absolute_error(y_train_comp, y_pred_train_comp):.2f} minutes")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_train_comp, y_pred_train_comp)):.2f} minutes")
print(f"  R²: {r2_score(y_train_comp, y_pred_train_comp):.3f}")

print(f"\nTest Set:")
print(f"  MAE: {mean_absolute_error(y_test_comp, y_pred_test_comp):.2f} minutes")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test_comp, y_pred_test_comp)):.2f} minutes")
print(f"  R²: {r2_score(y_test_comp, y_pred_test_comp):.3f}")
print(f"{'='*60}")

In [ ]:
# Feature importance for completion time
comp_importance = pd.DataFrame({
    'feature': completion_features,
    'importance': gb_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 5))
plt.barh(comp_importance['feature'], comp_importance['importance'], color='darkgreen')
plt.xlabel('Importance', fontsize=12)
plt.title(f'{WORKSHOP}: Feature Importance for Total Completion Time', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\nTop Features:")
print(comp_importance)

In [ ]:
# Visualize completion time predictions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot
axes[0].scatter(y_test_comp, y_pred_test_comp, alpha=0.6, s=80, color='purple')
axes[0].plot([y_test_comp.min(), y_test_comp.max()], 
             [y_test_comp.min(), y_test_comp.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Completion Time (min)', fontsize=11)
axes[0].set_ylabel('Predicted Completion Time (min)', fontsize=11)
axes[0].set_title('Completion Time: Predictions vs Actual', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Distribution comparison
axes[1].hist(y_test_comp, bins=15, alpha=0.5, label='Actual', color='blue')
axes[1].hist(y_pred_test_comp, bins=15, alpha=0.5, label='Predicted', color='orange')
axes[1].set_xlabel('Completion Time (min)', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
axes[1].set_title('Distribution: Actual vs Predicted', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Summary and Insights

In [ ]:
print("=" * 70)
print(f"PHASE 3 PREDICTIVE MODELING SUMMARY - {WORKSHOP}")
print("=" * 70)

print("\n🔮 Model 1: Time to Next Station")
print(f"  Algorithm: Random Forest Regressor")
print(f"  Training samples: {len(X_train)}")
print(f"  Test samples: {len(X_test)}")
print(f"  Test MAE: {mean_absolute_error(y_test, y_pred_test):.2f} minutes")
print(f"  Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f} minutes")
print(f"  Test R²: {r2_score(y_test, y_pred_test):.3f}")
print(f"  Most important feature: {feature_importance.iloc[0]['feature']}")

print("\n🎯 Model 2: Total Completion Time")
print(f"  Algorithm: Gradient Boosting Regressor")
print(f"  Training samples: {len(X_train_comp)}")
print(f"  Test samples: {len(X_test_comp)}")
print(f"  Test MAE: {mean_absolute_error(y_test_comp, y_pred_test_comp):.2f} minutes")
print(f"  Test RMSE: {np.sqrt(mean_squared_error(y_test_comp, y_pred_test_comp)):.2f} minutes")
print(f"  Test R²: {r2_score(y_test_comp, y_pred_test_comp):.3f}")
print(f"  Most important feature: {comp_importance.iloc[0]['feature']}")

print("\n" + "=" * 70)
print("💡 Recommendations:")
print("  1. Fine-tune hyperparameters for better performance")
print("  2. Add more features (environmental, group-specific)")
print("  3. Try advanced models (XGBoost, LightGBM, Neural Networks)")
print("  4. Implement cross-validation for robust evaluation")
print("  5. Deploy models for real-time predictions")
print("=" * 70)

## Save Phase 3 Results

In [ ]:
# Create output directory
output_dir = Path(f'../data/phase3_results/{WORKSHOP}')
output_dir.mkdir(parents=True, exist_ok=True)

# Save prediction features and results
pred_df['predicted_time_to_next'] = rf_model.predict(pred_df[feature_cols])
pred_df.to_csv(output_dir / 'next_station_predictions.csv', index=False)

completion_df['predicted_completion_time'] = gb_model.predict(completion_df[completion_features])
completion_df.to_csv(output_dir / 'completion_time_predictions.csv', index=False)

# Save feature importance
feature_importance.to_csv(output_dir / 'next_station_feature_importance.csv', index=False)
comp_importance.to_csv(output_dir / 'completion_feature_importance.csv', index=False)

# Save models
with open(output_dir / 'rf_next_station_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)

with open(output_dir / 'gb_completion_model.pkl', 'wb') as f:
    pickle.dump(gb_model, f)

# Save model performance metrics
metrics = {
    'workshop': WORKSHOP,
    'next_station_test_mae': mean_absolute_error(y_test, y_pred_test),
    'next_station_test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test)),
    'next_station_test_r2': r2_score(y_test, y_pred_test),
    'completion_test_mae': mean_absolute_error(y_test_comp, y_pred_test_comp),
    'completion_test_rmse': np.sqrt(mean_squared_error(y_test_comp, y_pred_test_comp)),
    'completion_test_r2': r2_score(y_test_comp, y_pred_test_comp)
}
pd.DataFrame([metrics]).to_csv(output_dir / 'model_metrics.csv', index=False)

print(f"✅ Phase 3 results saved to {output_dir}/")
print(f"\nSaved files:")
print(f"  • next_station_predictions.csv - Predictions for next station arrival")
print(f"  • completion_time_predictions.csv - Completion time predictions")
print(f"  • next_station_feature_importance.csv - Feature importance for Model 1")
print(f"  • completion_feature_importance.csv - Feature importance for Model 2")
print(f"  • rf_next_station_model.pkl - Trained Random Forest model")
print(f"  • gb_completion_model.pkl - Trained Gradient Boosting model")
print(f"  • model_metrics.csv - Performance metrics")
print(f"\n🎉 All three phases completed for {WORKSHOP}!")